In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
num_epochs = 5
train_batch_size = 20
test_batch_size = 10
learning_rate = 0.001
classes = [ 
    'T-shirt/top', 
    'Trouser', 
    'Pullover', 
    'Dress', 
    'Coat',
    'Sandal', 
    'Shirt', 
    'Sneaker',
    'Bag',
    'Ankle boot'
]

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

print(len(train_dataset))
print(len(test_dataset))

60000
10000


In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [6]:
def train(model, device, train_loader, optimizer, criterion, epochs, interval=500):
    for epoch in range(epochs):
        print(f'Starting #{epoch}')
        for batch_idx, (images, labels) in tqdm(enumerate(train_loader)):
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()

            if batch_idx % interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(images), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
        print(f'Finishing Epoch #{epoch}')

In [7]:
def test(model, device, test_loader):
    correct_imgs = 0
    matrix = []
    for _ in range(10):
        matrix.append([0] * 10)
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            correct_imgs += predictions.eq(labels.view_as(predictions)).sum().item()
            for j, (label, pred) in enumerate(list(zip(labels.tolist(), predictions.tolist()))):   
                matrix[pred][label] += 1
        
    return correct_imgs / len(test_loader.dataset), matrix

In [8]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)

In [9]:
train(model, device, train_loader, optimizer, criterion, 5, 100)

2it [00:00, 16.74it/s]

Starting #0
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.303894


104it [00:04, 24.24it/s]

Train Epoch: 0 [2000/60000 (3%)]	Loss: 0.698049


203it [00:08, 24.15it/s]

...

Train Epoch: 4 [56000/60000 (93%)]	Loss: 0.291412


2904it [01:58, 27.70it/s]

Train Epoch: 4 [58000/60000 (97%)]	Loss: 0.099110


3000it [02:01, 24.63it/s]

Finishing Epoch #4


In [10]:
test(model, device, test_loader)

100%|██████████| 1000/1000 [00:08<00:00, 116.57it/s]


(0.8722,
 [[872, 3, 22, 27, 1, 1, 195, 0, 4, 0],
  [0, 963, 0, 6, 2, 0, 1, 0, 2, 0],
  [17, 2, 819, 9, 101, 0, 101, 0, 4, 0],
  [35, 23, 12, 912, 42, 3, 32, 0, 5, 0],
  [6, 4, 75, 21, 772, 0, 73, 0, 3, 0],
  [2, 0, 1, 0, 0, 937, 0, 12, 1, 5],
  [55, 3, 65, 22, 78, 0, 583, 0, 8, 1],
  [0, 0, 0, 0, 0, 39, 0, 947, 5, 44],
  [13, 2, 6, 3, 4, 1, 15, 1, 968, 1],
  [0, 0, 0, 0, 0, 19, 0, 40, 0, 949]])